In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [309]:
predict = pd.read_csv("predict.csv")

In [310]:
predict.isna().sum()

Unnamed: 0    0
Cases         3
Death         3
Date_year     0
Date_month    0
Date_day      0
dtype: int64

In [311]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  646 non-null    int64  
 1   Cases       643 non-null    float64
 2   Death       643 non-null    float64
 3   Date_year   646 non-null    int64  
 4   Date_month  646 non-null    int64  
 5   Date_day    646 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 30.4 KB


In [312]:
import math
median_Cases = math.floor(predict.Cases.median())
median_death = math.floor(predict.Death.median())

predict.Death = predict.Death.fillna(median_death) 
predict.Cases = predict.Cases.fillna(median_Cases)  

In [313]:
predict.head()

,Unnamed: 0,Cases,Death,Date_year,Date_month,Date_day
0,0,4.0,0.0,2022,1,1
1,1,11.0,0.0,2022,1,2
2,2,29.0,0.0,2022,1,3
3,3,4.0,0.0,2022,1,4
4,4,4.0,0.0,2022,1,5


In [314]:
predict.isna().sum()

Unnamed: 0    0
Cases         0
Death         0
Date_year     0
Date_month    0
Date_day      0
dtype: int64

In [315]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [316]:
x = predict[['Date_day','Date_month','Date_year','Cases']]
y = predict.Death

In [317]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [318]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
            
        }
    }     
}

In [320]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

C:\Users\sayem\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\sayem\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\sayem\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\sayem\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\sayem\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has

,model,best_score,best_params
0,svm,0.640859,"{'C': 10, 'kernel': 'linear'}"
1,random_forest,0.519940,{'n_estimators': 5}
2,logistic_regression,0.620704,{'C': 1}
3,naive_bayes_gaussian,0.617567,{}
4,naive_bayes_multinomial,0.616017,{}
5,decision_tree,0.445498,{'criterion': 'entropy'}
